In [1]:
import random
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")



words = []
classes = []
documents = []
ignore_words = ["?", "!"]
data_file = open("intents.json").read()
intents = json.loads(data_file)


for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        
        w = nltk.word_tokenize(pattern)
        words.extend(w)
       
        documents.append((w, intent["tag"]))

        if intent["tag"] not in classes:
            classes.append(intent["tag"])


words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes")
print(classes)
print(len(words), "unique lemmatized words")

#pickle.dump(words, open("words.pkl", "wb"))
#pickle.dump(classes, open("classes.pkl", "wb"))

training = []
output_empty = [0] * len(classes)

for doc in documents:
   
    bag = []
 
    pattern_words = doc[0]
   
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
    

random.shuffle(training)
training = np.array(training)
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")


model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])


hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5", hist)
print("model created")


2024-04-25 19:02:49.175892: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 19:02:49.267414: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 19:02:49.272494: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 19:02:51.368815: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package omw-1.4 to /home/fatima/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/fatima/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/fat

108 documents
55 classes
['AI', 'abbr', 'artificial', 'bend', 'body', 'bot1', 'breathe', 'business', 'chatbot', 'chatterbox', 'clone', 'comp', 'computer', 'control', 'cramped', 'death', 'do', 'events', 'fight', 'goodbye', 'greetings', 'hardware', 'hobby', 'idea', 'imortal', 'lang', 'laugh', 'lie', 'machine', 'malfunction', 'motormouth', 'move', 'name', 'name1', 'need', 'noanswer', 'number', 'os', 'program', 'programming', 'ratchet', 'robotics', 'robots', 'robotss', 'sapient', 'sense', 'sentiment', 'shoe', 'sound', 'stupid', 'thanks', 'usage', 'who', 'wt', 'yes']
129 unique lemmatized words


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (108, 2) + inhomogeneous part.

In [3]:

import random
import numpy as np
import pickle
import json
from flask import Flask, render_template, request
#from flask_ngrok import run_with_ngrok
import nltk
from keras.models import load_model
from nltk.stem import WordNetLemmatizer
from binascii import a2b_base64
from werkzeug.utils import secure_filename
import urllib
from PIL import Image,ImageDraw
from keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt

model_img = load_model('./inceptionv3.h5')


    
class_names= ['Maltese Dog', 'Afghan Hound', 'Scottish Deerhound', 'Pomeranian', 
              'Irish Wolfhound', 'Bernese Mountain Dog', 'Samoyed', 'Shih-Tzu']    
    

lemmatizer = WordNetLemmatizer()


model_txt = load_model("chatbot_model.h5")
intents = json.loads(open("intents.json").read())
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

app = Flask(__name__)

from flask import redirect



@app.route("/")
def home():
    return render_template("index4.html")


@app.route("/get", methods=["POST"])


def chatbot_response():
    msg = request.form["msg"]
    img = request.form["img"]
    imgName = request.form["imgName"]
    imgExist = request.form["imgExists"]

    if imgExist=="true":
        response = urllib.request.urlopen(img)
        pic_name = "./uploads/image.jpg"
        
        with open(pic_name, 'wb') as f:
            f.write(response.file.read())
        
        pred =  predimage(pic_name)    
        
        res =  "Prediction is: "+pred+ "   real label: "+ imgName
    elif msg.startswith('my name is'):
         name = msg[11:]
         ints = predict_class(msg, model_txt)
         res1 = getResponse(ints, intents)
         res =res1.replace("{n}",name)
    elif msg.startswith('hi my name is'):
         name = msg[14:]
         ints = predict_class(msg, model_txt)
         res1 = getResponse(ints, intents)
         res =res1.replace("{n}",name)
    else:
         ints = predict_class(msg, model_txt)
         res = getResponse(ints, intents)
        
    return res


def predimage(path):
    
    test = load_img(path,target_size=(224,224))
    test = img_to_array(test)
    test = np.expand_dims(test,axis=0)
    test /= 255 
    result = model_img.predict(test,batch_size = 8)
    y_class = result.argmax(axis=-1)      
    result = list(np.around(np.array(result),2))
    i = y_class[0]
    s = result[0][i]
     
    return class_names[y_class[0]]



def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words



def bow(sentence, words, show_details=True):
    
    sentence_words = clean_up_sentence(sentence)
  
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


def predict_class(sentence, model_txt):
   
    p = bow(sentence, words, show_details=False)
    res = model_txt.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
 
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result



if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Apr/2024 19:03:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 19:03:26] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2024 19:03:27] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 135ms/step


127.0.0.1 - - [25/Apr/2024 19:03:35] "POST /get HTTP/1.1" 200 -


1/1 [==============================] - 0s 18ms/step


127.0.0.1 - - [25/Apr/2024 19:03:41] "POST /get HTTP/1.1" 200 -


1/1 [==============================] - 0s 18ms/step


127.0.0.1 - - [25/Apr/2024 19:03:48] "POST /get HTTP/1.1" 200 -


1/1 [==============================] - 2s 2s/step


127.0.0.1 - - [25/Apr/2024 19:04:42] "POST /get HTTP/1.1" 200 -
